## Groq Llama Model Invocation

In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="llama-3.1-70b-versatile"

)
response = llm.invoke("who is the second president of india")
print(response.content)

The second President of India was Sarvepalli Radhakrishnan.


## Web Scraping using WebBaseLoader

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://sevdesk.recruitee.com/o/data-scientist-mfx")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






















sevDesk GmbH - Data Scientist (m/f/x)




Zum Inhalt springensevDesk GmbHStartseiteKarriereseitesevDesk GmbHStartseiteKarriereseitesevDesk GmbHStartseiteKarriereseitesevDesk GmbHStartseiteData Scientist (m/f/x)Standort flexibelDeutschlandDataJobdetailsBewerbenDeine Rolle bei unsWe at sevdesk:
As an intuitive cloud accounting software, we empower founders, self-employed individuals, and small business owners to focus on what truly matters. Our product offers them a smart accounting tool to save time in their daily activities. As a company, we aim for excellence, continuously expand our horizons, and strive for daily improvement of our product and our daily work life. In pursuit of these goals, we rely on our strong culture, innovative spirit, and unwavering passion. In the future, we hope to count you among our dedicated team members.Your Mission:As a Data Scientist in our Data Team, you simplify accounting processes for our customers by designing and implementing pr

## Job Posting Extraction by Prompting Llama

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON as string of dictonary.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
print(type(res.content))

{
    "role": "Data Scientist (m/f/x)",
    "experience": "3-5 years of relevant professional experience as a Data Scientist",
    "skills": [
        "Machine learning/deep learning",
        "Statistical analysis",
        "SQL/NoSQL",
        "Python",
        "Common ML libraries (e.g., scikit-learn, pandas, numpy, PyTorch)",
        "Container technologies (such as Docker)",
        "Cloud platforms"
    ],
    "description": "As a Data Scientist in our Data Team, you simplify accounting processes for our customers by designing and implementing practical machine learning solutions that address real-world problems and deliver significant value for our users and our company. You collaborate with Product and ML engineering to seamlessly integrate your models into production systems."
}
<class 'str'>


## JASON Parsing

In [4]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
jparse = parser.parse(res.content)
print(jparse)
print(type(jparse))


{'role': 'Data Scientist (m/f/x)', 'experience': '3-5 years of relevant professional experience as a Data Scientist', 'skills': ['Machine learning/deep learning', 'Statistical analysis', 'SQL/NoSQL', 'Python', 'Common ML libraries (e.g., scikit-learn, pandas, numpy, PyTorch)', 'Container technologies (such as Docker)', 'Cloud platforms'], 'description': 'As a Data Scientist in our Data Team, you simplify accounting processes for our customers by designing and implementing practical machine learning solutions that address real-world problems and deliver significant value for our users and our company. You collaborate with Product and ML engineering to seamlessly integrate your models into production systems.'}
<class 'dict'>


## Loading Techstack and Links

In [5]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


## Chromadb 

In [6]:
import uuid
import chromadb

client = chromadb.PersistentClient('Vectorstore')
collection = client.get_or_create_collection(name = "portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

## Example to see how chromadb works

In [7]:
links = collection.query(query_texts=["Experience about ios"], n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [8]:
job = jparse
job['skills']

['Machine learning/deep learning',
 'Statistical analysis',
 'SQL/NoSQL',
 'Python',
 'Common ML libraries (e.g., scikit-learn, pandas, numpy, PyTorch)',
 'Container technologies (such as Docker)',
 'Cloud platforms']

In [9]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

## Prompting Llama to write cold email

In [10]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Seamless Integration

Dear Hiring Manager,

I came across the Data Scientist position at your esteemed organization and was impressed by the role's focus on designing and implementing practical machine learning solutions. As a Business Development Executive at AtliQ, I'd like to introduce our company as a potential partner in fulfilling your data science needs.

AtliQ is an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools. Our team of experts has empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our expertise in machine learning, deep learning, and statistical analysis aligns perfectly with the requirements of the Data Scientist role. We have a proven track record of delivering high-quality solutions using Python, common ML libraries (e.g., scikit-learn, pandas, 